# Strands Agents with AgentCore Memory (Long term memory) - MemoryManager

## Introduction

This tutorial demonstrates how to build an **intelligent customer support agent** using Strands agents integrated with AgentCore Memory via hooks using **MemoryManager** and **MemorySessionManager**. We'll focus on Long term memory for customer interaction history, remembering purchase details, and provides personalized support based on previous conversations and user preferences.

**NOTE: This is sample use MemoryManager & MemorySessionManager for implementation.**

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Long term Conversational                                                         |
| Agent type          | Customer Support                                                                 |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| Tutorial components | AgentCore Semantic and User Preferences Memory Extraction, Hooks for storing and retrieving Memory              |
| Example complexity  | Intermediate                                                                     |

You'll learn to:
- Set up AgentCore Memory with Long term strategies using MemoryManager
- Create memory hooks for automatic storage and retrieval with MemorySessionManager
- Build a customer support agent with persistent memory
- Handle customer issues with context from previous interactions

### Scenario Context
In this example, we will build a **Customer Support Use Case**. The agent will remember customer context, including order history, preferences, and previous issues, enabling more personalized and effective support. Conversations with customers are automatically stored using memory hooks, ensuring that important details are never lost. By employing multiple memory strategies such as semantic, and User Preference — the agent can capture a wide range of relevant information. This setup allows the agent to resolve issues with full awareness of the customer's history and preferences. Additionally, the agent is integrated with web search capabilities, making it easy to provide up-to-date product information and troubleshooting guidance as needed.

## Architecture

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## Prerequisites

To execute this tutorial you will need:
- Python 3.10+
- AWS credentials with Amazon Bedrock AgentCore Memory permissions
- Amazon Bedrock AgentCore SDK with MemoryManager support

## Step 1: Install Dependencies and Setup
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [1]:
!pip install -qr requirements.txt

In [4]:
import logging
import json
from typing import Dict, List
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Import required modules for Strands Agent
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS

# Import memory management modules
from bedrock_agentcore_starter_toolkit.operations.memory.manager import Memory, MemoryManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies import (
    SemanticStrategy, SummaryStrategy, CustomSemanticStrategy, CustomSummaryStrategy, CustomUserPreferenceStrategy,
    ExtractionConfig, ConsolidationConfig
)
from bedrock_agentcore.memory.constants import BlobMessage, ConversationalMessage, MessageRole, RetrievalConfig
from bedrock_agentcore.memory.session import Actor, MemorySession, MemorySessionManager
from bedrock_agentcore.memory.models import MemoryRecord

# Define message role constants
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

In [5]:
# Configuration - Replace with the correct values
REGION = "us-east-1"  
CUSTOMER_ID = "customer_001"
SESSION_ID = f"support_{datetime.now().strftime('%Y%m%d%H%M%S')}"

# Import boto3 for IAM role creation
import boto3
import json as json_module
from botocore.exceptions import ClientError

## Step 1.1: Create IAM Role for Custom Memory Strategies

Custom memory strategies require an execution role that allows AgentCore Memory to invoke Bedrock models for extraction and consolidation. This role is required when using `CustomSemanticStrategy` or `CustomUserPreferenceStrategy`.

In [6]:
# Create IAM role for AgentCore Memory custom strategies
def create_memory_execution_role():
    """Create IAM role for AgentCore Memory custom strategies with required permissions"""
    iam_client = boto3.client('iam', region_name=REGION)
    
    # Get current AWS account ID
    sts_client = boto3.client('sts', region_name=REGION)
    account_id = sts_client.get_caller_identity()['Account']
    
    role_name = "AgentCoreMemoryExecutionRole"
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # Trust policy for AgentCore Memory service
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "Service": [
                        "bedrock-agentcore.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": account_id
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock-agentcore:{REGION}:{account_id}:*"
                    }
                }
            }
        ]
    }
    
    # Permissions policy for Bedrock model invocation
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": [
                    "arn:aws:bedrock:*::foundation-model/*",
                    "arn:aws:bedrock:*:*:inference-profile/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": account_id
                    }
                }
            }
        ]
    }
    
    try:
        # Check if role already exists
        try:
            existing_role = iam_client.get_role(RoleName=role_name)
            logger.info(f"✅ IAM role already exists: {role_arn}")
            return role_arn
        except ClientError as e:
            if e.response['Error']['Code'] != 'NoSuchEntity':
                raise
        
        # Create the role
        logger.info(f"Creating IAM role: {role_name}")
        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json_module.dumps(trust_policy),
            Description="Execution role for AgentCore Memory custom strategies",
            Tags=[
                {
                    'Key': 'Purpose',
                    'Value': 'AgentCoreMemory'
                }
            ]
        )
        
        # Attach the permissions policy
        policy_name = "AgentCoreMemoryBedrockAccess"
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=policy_name,
            PolicyDocument=json_module.dumps(permissions_policy)
        )
        
        logger.info(f"✅ Successfully created IAM role: {role_arn}")
        logger.info(f"   - Trust policy: AgentCore Memory service can assume this role")
        logger.info(f"   - Permissions: bedrock:InvokeModel and bedrock:InvokeModelWithResponseStream")
        
        return role_arn
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'AccessDenied':
            logger.error("❌ Access denied creating IAM role. Please ensure you have IAM permissions:")
            logger.error("   - iam:CreateRole")
            logger.error("   - iam:PutRolePolicy")
            logger.error("   - iam:GetRole")
        else:
            logger.error(f"❌ Failed to create IAM role: {e}")
        raise
    except Exception as e:
        logger.error(f"❌ Unexpected error creating IAM role: {e}")
        raise

# Create the execution role
try:
    MEMORY_EXECUTION_ROLE_ARN = create_memory_execution_role()
    logger.info(f"✅ Memory execution role ready: {MEMORY_EXECUTION_ROLE_ARN}")
except Exception as e:
    logger.error(f"❌ Failed to create memory execution role: {e}")
    # For demo purposes, you can manually set the role ARN if creation fails
    # MEMORY_EXECUTION_ROLE_ARN = "arn:aws:iam::YOUR_ACCOUNT_ID:role/YourExistingRole"
    raise

2025-12-13 19:31:20,039 - INFO - Creating IAM role: AgentCoreMemoryExecutionRole
2025-12-13 19:31:20,373 - INFO - ✅ Successfully created IAM role: arn:aws:iam::455933813601:role/AgentCoreMemoryExecutionRole
2025-12-13 19:31:20,374 - INFO -    - Trust policy: AgentCore Memory service can assume this role
2025-12-13 19:31:20,374 - INFO -    - Permissions: bedrock:InvokeModel and bedrock:InvokeModelWithResponseStream
2025-12-13 19:31:20,377 - INFO - ✅ Memory execution role ready: arn:aws:iam::455933813601:role/AgentCoreMemoryExecutionRole


## Step 2: Create Memory Resource for Customer Support

For customer support, we'll use multiple memory strategies:
- **CustomUserPreferenceStrategy**: Captures customer preferences and behavior
- **CustomSemanticStrategy**: Stores order facts and product information

**IMPORTANT**: Custom strategies require an IAM execution role that allows AgentCore Memory to invoke Bedrock models. We created this role in Step 1.1 above.

In [7]:
# Initialize Memory Manager 
memory_manager = MemoryManager(region_name=REGION)
memory_name = "CustomerSupportLongTermMemory"

# Test basic memory manager initialization and connection
logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# Define memory strategies using typed classes
strategies = [
    CustomUserPreferenceStrategy(
        name="CustomerPreferences",
        description="Captures customer preferences and behavior",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract customer preferences and behavior patterns",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate customer preferences",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        namespaces=["support/customer/{actorId}/preferences"]
    ),
    CustomSemanticStrategy(
        name="CustomerSupportSemantic",
        description="Stores facts from conversations",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract factual information from customer support conversations",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate semantic insights from support interactions",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        namespaces=["support/customer/{actorId}/semantic"]
    )
]

# Validate strategy configuration
logger.info(f"✅ Configured {len(strategies)} memory strategies:")
for i, strategy in enumerate(strategies, 1):
    logger.info(f"  {i}. {strategy.name} ({type(strategy).__name__})")
    logger.info(f"     Description: {strategy.description}")
    logger.info(f"     Namespaces: {strategy.namespaces}")
    if hasattr(strategy, 'extraction_config') and strategy.extraction_config:
        logger.info(f"     Extraction Model: {strategy.extraction_config.model_id}")
    if hasattr(strategy, 'consolidation_config') and strategy.consolidation_config:
        logger.info(f"     Consolidation Model: {strategy.consolidation_config.model_id}"),

# Create memory resource using MemoryManager
logger.info(f"Creating memory '{memory_name}' with {len(strategies)} strategies...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=strategies,         # Pass typed strategy objects
        description="Memory for customer support agent",
        event_expiry_days=90,          # Memories expire after 90 days
        memory_execution_role_arn=MEMORY_EXECUTION_ROLE_ARN,  # Required for Custom strategies
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # Handle any errors during memory creation with enhanced error reporting
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # Cleanup on error - delete the memory if it was partially created
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # Re-raise the original exception
    raise

✅ MemoryManager initialized for region: us-east-1
2025-12-13 19:31:37,193 - INFO - ✅ MemoryManager initialized for region: us-east-1
2025-12-13 19:31:37,194 - INFO - ✅ MemoryManager initialized for region: us-east-1
2025-12-13 19:31:37,194 - INFO - Memory manager type: <class 'bedrock_agentcore_starter_toolkit.operations.memory.manager.MemoryManager'>
2025-12-13 19:31:37,195 - INFO - ✅ Configured 2 memory strategies:
2025-12-13 19:31:37,195 - INFO -   1. CustomerPreferences (CustomUserPreferenceStrategy)
2025-12-13 19:31:37,196 - INFO -      Description: Captures customer preferences and behavior
2025-12-13 19:31:37,196 - INFO -      Namespaces: ['support/customer/{actorId}/preferences']
2025-12-13 19:31:37,196 - INFO -      Extraction Model: global.anthropic.claude-haiku-4-5-20251001-v1:0
2025-12-13 19:31:37,197 - INFO -      Consolidation Model: global.anthropic.claude-haiku-4-5-20251001-v1:0
2025-12-13 19:31:37,197 - INFO -   2. CustomerSupportSemantic (CustomSemanticStrategy)
2025-

In [8]:
# Test memory manager basic functionality
try:
    # Test listing existing memories
    existing_memories = memory_manager.list_memories()
    logger.info(f"✅ Memory manager connection successful. Found {len(existing_memories)} existing memories")
    
    # Clean up any existing test memories
    for mem in existing_memories:
        if mem.name and mem.name.startswith("CustomerSupportLongTermMemory"):
            logger.info(f"Cleaning up existing test memory: {mem.id}")
            memory_manager.delete_memory(mem.id)
            
except Exception as e:
    logger.error(f"❌ Memory manager test failed: {e}")
    raise

2025-12-13 19:34:23,258 - INFO - ✅ Memory manager connection successful. Found 1 existing memories


Let's confirm if our memory contains the strategies we assigned

In [9]:
# Display memory information using MemoryManager
print(f"Memory ID: {memory.id}")
print(f"Memory Name: {memory.name}")
print(f"Memory Description: {memory.description}")
print(f"Memory Status: {memory.status}")
print(f"Number of strategies: {len(strategies)}")
for i, strategy in enumerate(strategies, 1):
    print(f"  {i}. {strategy.name}: {strategy.description}")

Memory ID: CustomerSupportLongTermMemory-hYevIXAdcF
Memory Name: CustomerSupportLongTermMemory
Memory Description: Memory for customer support agent
Memory Status: CREATING
Number of strategies: 2
  1. CustomerPreferences: Captures customer preferences and behavior
  2. CustomerSupportSemantic: Stores facts from conversations


## Step 3: Create Agent Tools

In [10]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for product information, troubleshooting guides, or support articles.
    
    Args:
        query: Search query for product info or troubleshooting
        max_results: Maximum number of results to return
    
    Returns:
        Search results with titles and snippets
    """
    try:
        results = DDGS().text(query, region="us-en", max_results=max_results)
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(f"{i}. {result.get('title', 'No title')}\n   {result.get('body', 'No description')}")
        
        return "\n".join(formatted_results)
    except RatelimitException:
        return "Rate limit reached: Please try again after a short delay."
    except DDGSException as d:
        return f"Search Error: {d}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

@tool
def check_order_status(order_number: str) -> str:
    """Check the status of a customer order.
    
    Args:
        order_number: The order number to check
    
    Returns:
        Order status information
    """
    # Simulate order lookup
    mock_orders = {
        "123456": "iPhone 15 Pro - Delivered on June 5, 2025",
        "654321": "Sennheiser Headphones - Delivered on June 25, 2025, 1-year warranty active",
        "789012": "Samsung Galaxy S23 - In transit, expected delivery on July 1, 2025",
    }
    
    return mock_orders.get(order_number, f"Order {order_number} not found. Please verify the order number.")

logger.info("✅ Check Order Status tool ready")

2025-12-13 19:34:49,686 - INFO - ✅ Web search tool ready
2025-12-13 19:34:49,688 - INFO - ✅ Check Order Status tool ready


## Step 4: Initialize Session Manager

**NEW: This section introduces the MemorySessionManager for session-based Memory operations.**

In [11]:
# Initialize the session memory manager
session_manager: MemorySessionManager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# Create a memory session for the specific customer
customer_session: MemorySession = session_manager.create_memory_session(
    actor_id=CUSTOMER_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Customer session created for actor: {CUSTOMER_ID}")
logger.info(f"   Session type: {type(customer_session)}")
logger.info(f"   Actor object: {customer_session.get_actor()}")

2025-12-13 19:34:57,986 - INFO - 💬 Creating new conversation for actor 'customer_001' in session 'support_20251213193115'...
2025-12-13 19:34:57,987 - INFO - ✅ Session manager initialized for memory: CustomerSupportLongTermMemory-hYevIXAdcF
2025-12-13 19:34:57,988 - INFO - ✅ Customer session created for actor: customer_001
2025-12-13 19:34:57,988 - INFO -    Session type: <class 'bedrock_agentcore.memory.session.MemorySession'>
2025-12-13 19:34:57,989 - INFO -    Actor object: {'actorId': 'customer_001'}


## Step 5: Create Memory Hook Provider for Customer Support
Hooks are special functions that run at specific points in an agent's execution lifecycle. Our custom hook provider will automatically manage customer support context by:
- **Saving support interactions** after each response using session-based methods
- **Retrieving and Injecting relevant context** from previous orders and preferences when processing new queries.


In [12]:
class CustomerSupportMemoryHooks(HookProvider):
    """Memory hooks for customer support agent - ENHANCED with MemorySession"""
    
    def __init__(self, customer_session: MemorySession):
        # Accept MemorySession directly 
        self.customer_session = customer_session
        
        # Define retrieval configuration for different memory types
        self.retrieval_config = {
            "support/customer/{actorId}/preferences": RetrievalConfig(top_k=3, relevance_score=0.3),
            "support/customer/{actorId}/semantic": RetrievalConfig(top_k=5, relevance_score=0.2)
        }
    
    def retrieve_customer_context(self, event: MessageAddedEvent):
        """Retrieve customer context before processing support query using MemorySession"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # Use MemorySession for context retrieval
                relevant_memories = []
                
                # Search across different memory namespaces using MemorySession
                for namespace_template, config in self.retrieval_config.items():
                    # Resolve namespace template with actual actor ID from session
                    resolved_namespace = namespace_template.format(
                        actorId=self.customer_session._actor_id
                    )
                    
                    # Use MemorySession API (no need to pass actor_id/session_id)
                    memories = self.customer_session.search_long_term_memories(
                        query=user_query,
                        namespace_prefix=resolved_namespace,
                        top_k=config.top_k
                    )
                    print(memories)
                    # Filter by relevance score
                    filtered_memories = [
                        memory for memory in memories
                        if memory.get("score", 0) >= config.relevance_score
                    ]
                    
                    relevant_memories.extend(filtered_memories)
                    logger.info(f"Found {len(filtered_memories)} relevant memories in {resolved_namespace}")
                
                # Inject context into agent's system prompt if memories found
                if relevant_memories:
                    context_text = self._format_context(relevant_memories)
                    original_prompt = event.agent.system_prompt
                    enhanced_prompt = f"{original_prompt}\n\nCustomer Context:\n{context_text}"
                    event.agent.system_prompt = enhanced_prompt
                    logger.info(f"✅ Injected {len(relevant_memories)} memories into agent context")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")
    
    def _format_context(self, memories: List[MemoryRecord]) -> str:
        """Format retrieved memories for agent context"""
        context_lines = []
        for i, memory in enumerate(memories[:5], 1):  # Limit to top 5
            content = memory.get('content', {}).get('text', 'No content available')
            score = memory.get('score', 0)
            context_lines.append(f"{i}. (Score: {score:.2f}) {content[:200]}...")
        
        return "\n".join(context_lines)
    
    def save_support_interaction(self, event: AfterInvocationEvent):
        """Save support interaction using MemorySession (cleaner API)"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last customer query and agent response
                customer_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                if customer_query and agent_response:
                    # Use MemorySession (no need to pass actor_id/session_id)
                    interaction_messages = [
                        ConversationalMessage(customer_query, USER),
                        ConversationalMessage(agent_response, ASSISTANT)
                    ]
                    
                    result = self.customer_session.add_turns(interaction_messages)
                    logger.info(f"✅ Saved interaction using MemorySession - Event ID: {result['eventId']}")
                    
        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register customer support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)  # Re-added!
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("✅ Customer support memory hooks registered with MemorySession")
print('Executed!')

Executed!


### Step 6: Create Customer Support Agent

In [13]:
# Create memory hooks using MemorySession
support_hooks = CustomerSupportMemoryHooks(customer_session)

# Create customer support agent
support_agent = Agent(
    hooks=[support_hooks],
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[web_search, check_order_status],
    system_prompt="""You are a helpful customer support agent with access to customer history and order information. 
    
    Your role:
    - Help customers with their orders, returns, and product issues
    - Use customer context to provide personalized support
    - Search for product information when needed
    - Be empathetic and solution-focused
    - Reference previous orders and preferences when relevant
    
    Always be professional, helpful, and aim to resolve customer issues efficiently."""
)

print("✅ Customer support agent created with MemorySession integration")

2025-12-13 19:35:29,579 - INFO - ✅ Customer support memory hooks registered with MemorySession


✅ Customer support agent created with MemorySession integration


### Step 7: Seed Customer History

Let's add some previous customer interactions to demonstrate memory functionality.

**NOTE: This section uses ConversationalMessage format and session-based storage.**

In [14]:
# Seed with previous customer interactions using MemorySession
previous_interactions = [
    ConversationalMessage("I bought a new iPhone 15 Pro on June 1st, 2025. Order number is 123456.", USER),
    ConversationalMessage("Thank you for your purchase! I can see your iPhone 15 Pro order #123456 was delivered successfully. How can I help you today?", ASSISTANT),
    ConversationalMessage("I also ordered Sennheiser headphones on June 20th. Order number 654321. They came with 1-year warranty.", USER),
    ConversationalMessage("Perfect! I have your Sennheiser headphones order #654321 on file with the 1-year warranty. Both your iPhone and headphones should work great together.", ASSISTANT),
    ConversationalMessage("I'm looking for a good laptop. I prefer ThinkPad models.", USER),
    ConversationalMessage("Great choice! ThinkPads are excellent for their durability and performance. Let me help you find the right model for your needs.", ASSISTANT)
]

# Save using MemorySession 
try:
    event_response = customer_session.add_turns(previous_interactions)
    logger.info(f"✅ Seeded customer history using MemorySession")
    logger.info(f"   Event ID: {event_response['eventId']}")
except Exception as e:
    logger.error(f"⚠️ Error seeding history: {e}")


2025-12-13 19:35:35,985 - INFO -   -> Storing 6 messages in short-term memory...
2025-12-13 19:35:36,352 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 19:35:36,353 - INFO - ✅ Seeded customer history using MemorySession
2025-12-13 19:35:36,353 - INFO -    Event ID: 0000001765654535987#5830b89e


#### Agent is ready to go. 

### Lets test Customer Support Scenarios

In [15]:
# Test 1: Customer reports iPhone issue 
logger.info("🧪 Running Test 1: iPhone performance issue")
test_query_1 = "My iPhone is running very slow and gets hot when charging. Can you help?"
logger.info(f"Query: {test_query_1}")

response1 = support_agent(test_query_1)
logger.info(f"✅ Test 1 completed successfully")
print(f"\n📱 iPhone Issue Support Response:\n{response1}\n")

2025-12-13 19:35:46,184 - INFO - 🧪 Running Test 1: iPhone performance issue
2025-12-13 19:35:46,185 - INFO - Query: My iPhone is running very slow and gets hot when charging. Can you help?
2025-12-13 19:35:46,187 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/preferences' with query: 'My iPhone is running very slow and gets hot when charging. Can you help?'...
2025-12-13 19:35:46,573 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:35:46,573 - INFO - Found 0 relevant memories in support/customer/customer_001/preferences
2025-12-13 19:35:46,574 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/semantic' with query: 'My iPhone is running very slow and gets hot when charging. Can you help?'...


[]


2025-12-13 19:35:46,901 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:35:46,902 - INFO - Found 0 relevant memories in support/customer/customer_001/semantic
2025-12-13 19:35:46,902 - INFO - Creating Strands MetricsClient


[]
I'd be happy to help you troubleshoot your iPhone! Let me search for information about slow performance and heat issues while charging.
Tool #1: web_search


2025-12-13 19:35:50,707 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=iPhone%20running%20slow%20and%20hot%20when%20charging%20troubleshooting 200
2025-12-13 19:35:50,711 - INFO - response: https://search.yahoo.com/search;_ylt=sBEXplmact6g3A7Cv4dd8auy;_ylu=ujXRf-at_87FuMpieAd3f8FzxOgWYV2uXe_qIyY3jXA83nE?p=iPhone+running+slow+and+hot+when+charging+troubleshooting 200


Great! I found some helpful information. Here are some common causes and solutions for your iPhone running slow and getting hot while charging:

**Common Causes:**
- **Fast charging heat**: Using a 20W or higher charger generates heat as a byproduct of rapid energy transfer (this is somewhat normal)
- **Background processes**: Resource-intensive apps running in the background
- **Battery or charging issues**: Could indicate a battery that needs replacement
- **Software issues**: Outdated iOS or apps

**Here are some troubleshooting steps you can try:**

1. **While charging:**
   - Remove your iPhone case to allow better heat dissipation
   - Stop using resource-intensive apps while charging
   - Avoid using your phone while it's charging if possible
   - Keep it away from direct sunlight

2. **General optimization:**
   - Enable **Low Power Mode** (Settings > Battery)
   - Update your iPhone to the latest iOS version
   - Update all apps to their latest versions
   - Close background a

2025-12-13 19:36:00,783 - INFO -   -> Storing 2 messages in short-term memory...


 will help me provide more specific advice. Also, is the heat excessive or just noticeable warmth?

2025-12-13 19:36:01,056 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 19:36:01,057 - INFO - ✅ Saved interaction using MemorySession - Event ID: 0000001765654560784#100b41d4
2025-12-13 19:36:01,059 - INFO - ✅ Test 1 completed successfully



📱 iPhone Issue Support Response:
Great! I found some helpful information. Here are some common causes and solutions for your iPhone running slow and getting hot while charging:

**Common Causes:**
- **Fast charging heat**: Using a 20W or higher charger generates heat as a byproduct of rapid energy transfer (this is somewhat normal)
- **Background processes**: Resource-intensive apps running in the background
- **Battery or charging issues**: Could indicate a battery that needs replacement
- **Software issues**: Outdated iOS or apps

**Here are some troubleshooting steps you can try:**

1. **While charging:**
   - Remove your iPhone case to allow better heat dissipation
   - Stop using resource-intensive apps while charging
   - Avoid using your phone while it's charging if possible
   - Keep it away from direct sunlight

2. **General optimization:**
   - Enable **Low Power Mode** (Settings > Battery)
   - Update your iPhone to the latest iOS version
   - Update all apps to their lates

In [16]:
# Test 2: Bluetooth connectivity issue 
logger.info("🧪 Running Test 2: Bluetooth connectivity issue")
test_query_2 = "My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?"
logger.info(f"Query: {test_query_2}")

response2 = support_agent(test_query_2)
logger.info(f"✅ Test 2 completed successfully")
print(f"\n🎧 Bluetooth Issue Support Response:\n{response2}\n")

2025-12-13 19:36:06,775 - INFO - 🧪 Running Test 2: Bluetooth connectivity issue
2025-12-13 19:36:06,776 - INFO - Query: My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?
2025-12-13 19:36:06,778 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/preferences' with query: 'My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?'...
2025-12-13 19:36:07,093 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:36:07,094 - INFO - Found 0 relevant memories in support/customer/customer_001/preferences
2025-12-13 19:36:07,095 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/semantic' with query: 'My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?'...


[]


2025-12-13 19:36:07,370 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:36:07,371 - INFO - Found 0 relevant memories in support/customer/customer_001/semantic


[]
Let me search for troubleshooting steps to help you reconnect your iPhone to your Sennheiser headphones via Bluetooth.
Tool #2: web_search


2025-12-13 19:36:09,387 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=iPhone%20Sennheiser%20headphones%20Bluetooth%20connection%20issues%20troubleshooting 200
2025-12-13 19:36:09,947 - INFO - response: https://search.brave.com/search?q=iPhone+Sennheiser+headphones+Bluetooth+connection+issues+troubleshooting&source=web 200


Good news! I found some troubleshooting steps. Here's how to fix your iPhone and Sennheiser headphones Bluetooth connection:

**Basic Troubleshooting Steps:**

1. **Reset the Bluetooth connection:**
   - Go to Settings > Bluetooth
   - Find your Sennheiser headphones in the list
   - Tap the "i" icon next to them
   - Select "Forget This Device"

2. **Put your Sennheiser headphones in pairing mode:**
   - Turn off your headphones
   - Press and hold the power button until the indicator blinks white
   - Release to enter manual pairing mode
   - Some models may have a specific pairing button - check your manual

3. **Re-pair on your iPhone:**
   - On your iPhone, go to Settings > Bluetooth
   - Make sure Bluetooth is turned ON
   - Your Sennheiser headphones should appear in the Available Devices list
   - Tap on them to connect

4. **If they still don't appear:**
   - Restart your iPhone (turn it off and back on)
   - Restart your Sennheiser headphones
   - Move closer to your iPhone t

2025-12-13 19:36:15,747 - INFO -   -> Storing 2 messages in short-term memory...


, or is this the first time you're trying to pair them?

2025-12-13 19:36:16,018 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 19:36:16,018 - INFO - ✅ Saved interaction using MemorySession - Event ID: 0000001765654575748#6caba763
2025-12-13 19:36:16,020 - INFO - ✅ Test 2 completed successfully



🎧 Bluetooth Issue Support Response:
Good news! I found some troubleshooting steps. Here's how to fix your iPhone and Sennheiser headphones Bluetooth connection:

**Basic Troubleshooting Steps:**

1. **Reset the Bluetooth connection:**
   - Go to Settings > Bluetooth
   - Find your Sennheiser headphones in the list
   - Tap the "i" icon next to them
   - Select "Forget This Device"

2. **Put your Sennheiser headphones in pairing mode:**
   - Turn off your headphones
   - Press and hold the power button until the indicator blinks white
   - Release to enter manual pairing mode
   - Some models may have a specific pairing button - check your manual

3. **Re-pair on your iPhone:**
   - On your iPhone, go to Settings > Bluetooth
   - Make sure Bluetooth is turned ON
   - Your Sennheiser headphones should appear in the Available Devices list
   - Tap on them to connect

4. **If they still don't appear:**
   - Restart your iPhone (turn it off and back on)
   - Restart your Sennheiser headpho

In [18]:
# Test 3: Check order status 
logger.info("🧪 Running Test 3: Order status check")
test_query_3 = "Can you check the status of my recent orders?"
logger.info(f"Query: {test_query_3}")

response3 = support_agent(test_query_3)
logger.info(f"✅ Test 3 completed successfully")
print(f"\n📦 Order Status Support Response:\n{response3}\n")

2025-12-13 19:36:48,519 - INFO - 🧪 Running Test 3: Order status check
2025-12-13 19:36:48,520 - INFO - Query: Can you check the status of my recent orders?
2025-12-13 19:36:48,522 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/preferences' with query: 'Can you check the status of my recent orders?'...
2025-12-13 19:36:48,778 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:36:48,778 - INFO - Found 0 relevant memories in support/customer/customer_001/preferences
2025-12-13 19:36:48,779 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/semantic' with query: 'Can you check the status of my recent orders?'...


[]


2025-12-13 19:36:49,018 - INFO -      ✅ Found 2 relevant long-term records.
2025-12-13 19:36:49,019 - INFO - Found 2 relevant memories in support/customer/customer_001/semantic
2025-12-13 19:36:49,019 - INFO - ✅ Injected 2 memories into agent context


[{'memoryRecordId': 'mem-fde55cc5-a827-471e-9406-90e3e62ee17e', 'content': {'text': 'The user purchased Sennheiser headphones on June 20th with order number 654321 and a 1-year warranty.'}, 'memoryStrategyId': 'CustomerSupportSemantic-97Pg9kGbvi', 'namespaces': ['support/customer/customer_001/semantic'], 'createdAt': datetime.datetime(2025, 12, 13, 19, 35, 35, 987000, tzinfo=tzlocal()), 'score': 0.38175985, 'metadata': {'x-amz-agentcore-memory-recordType': {'stringValue': 'BASE'}}}, {'memoryRecordId': 'mem-15e77dae-675b-4bff-bd05-f2f08dedfa58', 'content': {'text': 'The user purchased an iPhone 15 Pro on June 1st, 2025 with order number 123456.'}, 'memoryStrategyId': 'CustomerSupportSemantic-97Pg9kGbvi', 'namespaces': ['support/customer/customer_001/semantic'], 'createdAt': datetime.datetime(2025, 12, 13, 19, 35, 35, 987000, tzinfo=tzlocal()), 'score': 0.36706156, 'metadata': {'x-amz-agentcore-memory-recordType': {'stringValue': 'BASE'}}}]
Of course! Let me check the status of both your

2025-12-13 19:36:53,523 - INFO -   -> Storing 2 messages in short-term memory...


 troubleshooting for either product?

2025-12-13 19:36:53,712 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 19:36:53,712 - INFO - ✅ Saved interaction using MemorySession - Event ID: 0000001765654613524#12451914
2025-12-13 19:36:53,714 - INFO - ✅ Test 3 completed successfully



📦 Order Status Support Response:
Great news! Here's the status of your recent orders:

**Order #123456 - iPhone 15 Pro**
- Status: ✅ **Delivered on June 5, 2025**

**Order #654321 - Sennheiser Headphones**
- Status: ✅ **Delivered on June 25, 2025**
- Warranty: **1-year warranty active**

Both of your items have been successfully delivered! Since you're experiencing issues with both devices (the iPhone running slow/getting hot and the Bluetooth connectivity problem with the headphones), here are some options:

1. **For your iPhone 15 Pro**: If the overheating and slow performance persist after trying the troubleshooting steps I mentioned, we can look into warranty service or a replacement.

2. **For your Sennheiser Headphones**: Since your 1-year warranty is active, if we can't resolve the Bluetooth connection issue through troubleshooting, you may be eligible for warranty service or replacement.

Would you like me to help you with any warranty claims or additional troubleshooting for 

In [19]:
# Test 4: Product recommendation based on preferences 
logger.info("🧪 Running Test 4: Product recommendation")
test_query_4 = "I'm still interested in buying a laptop. What ThinkPad models do you recommend?"
logger.info(f"Query: {test_query_4}")

response4 = support_agent(test_query_4)
logger.info(f"✅ Test 4 completed successfully")
print(f"\n💻 Product Recommendation Support Response:\n{response4}\n")

logger.info("🎉 All customer support scenario tests completed!")

2025-12-13 19:37:00,288 - INFO - 🧪 Running Test 4: Product recommendation
2025-12-13 19:37:00,289 - INFO - Query: I'm still interested in buying a laptop. What ThinkPad models do you recommend?
2025-12-13 19:37:00,291 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/preferences' with query: 'I'm still interested in buying a laptop. What ThinkPad models do you recommend?'...
2025-12-13 19:37:00,524 - INFO -      ✅ Found 0 relevant long-term records.
2025-12-13 19:37:00,525 - INFO - Found 0 relevant memories in support/customer/customer_001/preferences
2025-12-13 19:37:00,525 - INFO -   -> Querying long-term memory in namespace 'support/customer/customer_001/semantic' with query: 'I'm still interested in buying a laptop. What ThinkPad models do you recommend?'...


[]


2025-12-13 19:37:00,787 - INFO -      ✅ Found 2 relevant long-term records.
2025-12-13 19:37:00,787 - INFO - Found 2 relevant memories in support/customer/customer_001/semantic
2025-12-13 19:37:00,788 - INFO - ✅ Injected 2 memories into agent context


[{'memoryRecordId': 'mem-fde55cc5-a827-471e-9406-90e3e62ee17e', 'content': {'text': 'The user purchased Sennheiser headphones on June 20th with order number 654321 and a 1-year warranty.'}, 'memoryStrategyId': 'CustomerSupportSemantic-97Pg9kGbvi', 'namespaces': ['support/customer/customer_001/semantic'], 'createdAt': datetime.datetime(2025, 12, 13, 19, 35, 35, 987000, tzinfo=tzlocal()), 'score': 0.3539441, 'metadata': {'x-amz-agentcore-memory-recordType': {'stringValue': 'BASE'}}}, {'memoryRecordId': 'mem-15e77dae-675b-4bff-bd05-f2f08dedfa58', 'content': {'text': 'The user purchased an iPhone 15 Pro on June 1st, 2025 with order number 123456.'}, 'memoryStrategyId': 'CustomerSupportSemantic-97Pg9kGbvi', 'namespaces': ['support/customer/customer_001/semantic'], 'createdAt': datetime.datetime(2025, 12, 13, 19, 35, 35, 987000, tzinfo=tzlocal()), 'score': 0.34356287, 'metadata': {'x-amz-agentcore-memory-recordType': {'stringValue': 'BASE'}}}]
I appreciate you asking again, but as I mentione

2025-12-13 19:37:04,105 - INFO -   -> Storing 2 messages in short-term memory...
2025-12-13 19:37:04,351 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 19:37:04,352 - INFO - ✅ Saved interaction using MemorySession - Event ID: 0000001765654624107#3f048bb7
2025-12-13 19:37:04,353 - INFO - ✅ Test 4 completed successfully
2025-12-13 19:37:04,354 - INFO - 🎉 All customer support scenario tests completed!



💻 Product Recommendation Support Response:
I appreciate you asking again, but as I mentioned before, laptop recommendations are outside my scope as a customer support agent. My role is to help with:

- **Your existing orders and products**
- **Troubleshooting issues** with devices you own
- **Returns, replacements, and warranty support**
- **Order status and tracking**

For ThinkPad recommendations, I'd suggest checking tech review sites, the Lenovo website, or contacting our sales team.

**However, is there anything I can still help you with regarding your current issues?** We were working on:

1. **iPhone 15 Pro** - slow performance and overheating while charging
2. **Sennheiser Headphones** - Bluetooth connection issues

Have you had a chance to try the troubleshooting steps I provided? If the problems persist, we can explore warranty options since both products are still covered. Let me know how I can assist!




#### Customer Support Tutorial completed! 🎉 
Key takeaways:
- Memory hooks automatically manage customer context across support sessions using MemorySessionManager
- Multi-strategy memory captures orders, preferences, and facts from conversations using typed strategy classes
- Agents can provide personalized support based on customer history
- Tools can be integrated for order lookup and web search functionality
- Customer support becomes more efficient with persistent memory

## Clean Up

### Optional: Delete Memory Resource

In [ ]:
# Uncomment to delete the memory resource using MemoryManager
# try:
#     memory_manager.delete_memory(memory_id)
#     print(f"✅ Deleted memory resource using MemoryManager: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")